# reducing the size of Word2vec model

inspired by and English word data taken from : https://github.com/eyaler/word2vec-slim

download the word list files to `dicts` dir from https://github.com/eyaler/word2vec-slim/tree/master/source/dicts

In [6]:
from gensim.models import Word2Vec, KeyedVectors
import time
import numpy as np
import gzip
import os

In [2]:
# laod word directories (english words + our topic words)

words = set()
for dict_filename in os.listdir('dicts'):
    with gzip.open('dicts/'+dict_filename, 'rt', encoding='utf8') as f:
        temp = f.readlines()
        save_len = len(temp)
        for i in range(len(temp)):
            temp[i] = temp[i].strip().lower()
        temp = set(temp)
        print('%s: %d -> %d' % (dict_filename, save_len, len(temp)))
    words |= temp
print('combined: %d' % (len(words)))

words2.txt.gz: 235886 -> 234371
urban50.txt.gz: 86724 -> 86724
words.txt.gz: 354986 -> 354984
words3.txt.gz: 479829 -> 462984
topic_list_slim-model_26396topics_topn10_dep5_thr0.5.txt.gz: 26396 -> 26396
combined: 546117


In [29]:
# load the big model 

start = time.time()
model = KeyedVectors.load_word2vec_format('/home/dzon/kajo/jiri_w2v', binary=True)
print('Finished loading original model %.2f min' % ((time.time()-start)/60))


Finished loading original model 0.79 min


In [21]:
# how much memory the model occupies (~4GB)

from pympler import asizeof
asizeof.asizeof(model)

3936720192

In [22]:
print('word2vec total words: %d' % len(model.vocab))
print('non-phrases: %d' % len([w for w in model.vocab.keys() if '_' not in w]))
print('phrases: %d' % len([w for w in model.vocab.keys() if '_' in w]))

word2vec total words: 2556470
non-phrases: 1355029
phrases: 1201441


In [23]:
max_suffix_len = 2
min_base_len = 8

indices_to_delete = []
j = 0
suffix_grace_words = 0
for i,w in enumerate(model.index2word):
    l = w.strip().lower()
    found = False
    if l in words:
        found = True
    else:
        for s in range(1, 1+max_suffix_len):
            if len(l)-s<min_base_len:
                break
            elif l[:-s] in words:
                suffix_grace_words += 1
                found = True
                break

    if found:
        model.vocab[w].index = j
        j += 1
    else:
        del model.vocab[w]
        indices_to_delete.append(i)

model.syn0 = np.delete(model.syn0, indices_to_delete, axis=0)
print('slim: %d' % len(model.vocab))
print('suffix grace words: %d' % (suffix_grace_words))


/home/dzon/env/lib/python3.6/site-packages/ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).


slim: 196944
suffix grace words: 22349


In [24]:
slim_model_name = 'w2v-jiri-slim.bin'
model.save_word2vec_format(slim_model_name, binary=True)


In [25]:
del model

start = time.time()
model = KeyedVectors.load_word2vec_format(slim_model_name, binary=True)
print('Finished loading slim model %.1f sec' % ((time.time()-start)))

Finished loading slim model 3.4 sec


In [26]:
# how much memory the model occupies (~300MB)

from pympler import asizeof
asizeof.asizeof(model)

306357792

In [28]:
wlist = list(words)
wlist[:20]

['',
 'disproportionable',
 'psychophobia',
 'rowlandite',
 'unmedalled',
 'godmaker',
 'snottie',
 'unagricultural',
 'usitative',
 "bachelor's-buttons",
 'specialised/minority',
 'puppetlike',
 'resp.',
 'strap-on',
 'sing-song',
 'pre-employment',
 'contradictedness',
 'nonvagrantly',
 'macbitch',
 'cubonavicular']